<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/word2vec_%EC%86%8D%EB%8F%84_%EA%B0%9C%EC%84%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## word2vec 개선 1
- CBOW 모델이 다룰 말뭉치의 어휘수가  커지게 되면 몇 가지 문제가 발생
  - 예) 어휘 100만개, 은닉층의 뉴런이 100개인 CBOW 모델
  - 입력층과 출력층에 각 100만개 뉴런이 존재하게되는데 다음 두 계산에서 병목현상이 일어나게 됨
  
    ㄱ. 입력층의 원핫 표현과 가중치 행렬 W_in의 곱 계산
    - 단어를 원핫 표현으로 변환하여 다루기 때문에 어휘 수가 증가하면 원핫 표현으로 변환한 벡터의 크기도 증가하게 됨. 
    - 총 어휘의 수가 100만개라면 한 단어의 원핫 벡터의 원소수가 100만개가 됨. 상당한 메모리를 차지하게 됨
    - 엄청난 크기의 원핫 벡터와 가중치 행렬을 곱할 때 역시 상당한 계산 자원을 사용하게 됨
    - 이 문제를 해결하기 위해 Embedding 계층을 도입함

    ㄴ. 은닉층과 가중치 행렬 W_out의 곱 및 softmax  계층의 계산
    - 은닉층과 구장치 행렬의 곱만 해도 많은 계산량을 요구하게 됨
    - Softmax 계층에서도 다루는 어휘가 증가하면 계산량이 증가함
    - 이 문제를 해결하기 위해 네거티브 샘플링이란 새로운 손실 함수를 도입함
    

### Embedding 계층
- 단어의 원핫 표현과 가중치 행렬의 곱 결과는 특정 행렬의 특정 행을 추출하는 것과 같음
- 따라서, 가중치 매개변수로부터 '단어ID'에 해당하는 행벡터를 추출하는 계층을 만들어 이를 Embedding 계층이라고 명명
   - Embedding이란 단어 임베딩(word embedding)에서 유래한 것으로 자연어 처리 분야에서 단어의 밀집벡터 표현을 단어 임베딩 또는 단어의 분산 표현이라 함

In [2]:
# 행렬에서 특정 행을 추출하는 방법
import numpy as np

W = np.arange(21).reshape(7,3)
print(W)
print(W[2])
print(W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]


In [3]:
# 가중치 W에서 여러 행을 한번에 추출하는 방법
idx = np.array([1,0,3,0])
print(W[idx])

[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [5]:
class Embedding :
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None # 추출하는 행의 인덱스(단어 ID)를 가진 배열을 초기화
  
  # 순젆파 : 입력인 단어 벡터 행에서 해당 단어에 헤당하는 가중치 행을 추출하는 것
  def forward(self, idx) :
    W, = self.params
    self.idx = idx
    out = W[idx] 
    return out

  # 역전파 : 출력 층으로 부터 전해진 기울기를 다음 입력층으로 그대로 흘려보내는 것임
  # 유의 : 구해진 기울기는 앞 층으로부터 전해진 가중치의 특정 행에 설정됨
  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0 # dW의 형상을 유지한 채 해당 메모리에 덮어쓰는 방식. 이미 구현해둔 옵티마이저 클래스와 조합하는 것을 고려하여 이렇게 구현
    
    # 만약 idx 배열 원소 중 중복된 번호가 존재한다면 행을 단순 할당해서는 안됨
    # 은닉층의 뉴런은 순전파 시 여러 개로 복사되는 Repeat 노드에 해당하므로 
    # 역전파 시에는 여러 개의 기울기 값을 더해줌
    # for i, word_in enumerate(self.idx) :
    #   dW[word_id] += dout[i]

    # for문 대신 np.add.at(A, idx, B) 가능 --> B를 A의 idx번째 행에 더해주는 넘파이의 메소드
    np.add.at(dW, self.idx, dout)
    return None

## word2vec 개선 2
- 네거티브 샘플링(부정적 샘플링) 기법을 사용하여 계산량을 낮은 수준에서 일정하게 억제

### 은닉층 이후 계산에서의 문제점
- 거대한 행렬의 곱셈 시 많은 양의 시간과 메모리가 소요됨
- Softmax에서도 어휘 수만큼의 exp 계산이 수행되어야 하기 대문에 어휘 수에 비례하여 계샨량이 증가함

### 네거티브 샘플링
- 다중 분류 혹은 다중 클래스 분류 문제를 이진 분류 문제로 근사하는 것이 핵심
  - '맥락이 you와 goodbye일 때, 타깃 단어는 무엇인가?'라는 문제를 '맥락이 you와 goodbye일때, 타깃 단어는 sat입니까?'라는 예/아니오 문제에 대한 해결을 하도록 하는 것
  - 이진 분류 문제로 근사할 경우, 출력층에서는 뉴런 하나만 필요하게 되며, 이 출력층의 뉴런은 'say'의 점수를 출력하면 됨
  - 은닉층과 출력층의 가중치 행렬의 내적은 'say'에 해당하는 열(단어 벡터)만 추출하고, 그 추출된 벡터와 은닉층 뉴런과 내적을 계산하면 됨.
  - 이전까지의 출력층에서는 모든 단어에 대한 계산을 수행하였으나, 여기에서는 알고자 하는 단어에 주목하여 그 점수만을 계산하는 것이 다름


### 시그모이드 함수와 교차 엔트로피 오차
- 신경망으로 이진 분류 문제를 해결하고자 하는 경우엔  출력된 점수에 시그모이드 함수를 적용하여 확률로 변환하고, 손실을 구할 때 교차 엔트로피 오차를 사용함

  - 다중 분류의 경우, 출력층에서는 (점수를 확률로 변환 시) 소프트맥스 함수를 사용하고, 손실 함수로 교차 엔트로피 오차를 이용

- 시그모이드 함수 
  - S자 곡선 형태
  - 입력 값은 0~1사이의 실수로 변환됨
  - 시그모이드 함수의 출력(y)을 '확률'로 해석할 수 있음

- 교차 엔트로피 오차
  - 정답 레이블 값이 1('yes')이면 -logy이고, 0('no')이면 -log(1-y)가 출력
  
- 시그모이드 함수와 교차 엔트로피 오차를 조합한 역전파의 값이 y-t임
- 소프트맥스 함수와 교차 엔트로피 오차의 조합 또는 항등 함수와 2제곱 오차의 조합 역시 역전파 시에는 y-t 값이 전파됨

In [6]:
class EmbeddingDot : 
  def __init__(self, W) :
    self.embed = Embedding(W) # 
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None # 순전파 시의 계산 결과를 잠시 유지하기 위한 변수

  def forward(self, h, idx) :
    target_W = self.embed.forward(idx)  # 해당 단어의 분산표현 벡터
    out = np.sum(target_W * h, axis =1) # 점수 추출

    self.cache = (h, target_W) # 은닉층 뉴런의 값과 해당 단어의 벡터
    return out

  def backward(self, dout) :
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0], 1)

    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

### 네거티브 샘플링
- 위의 구현으로 주어진 문제를 다중 분류에서 이진 분류로 변환했으나. 긍정적인 예(쩡답)에 대해서만 학습하였으므로 오답에 대한 학습이 이루어지지 않은 상태
- 하지만 모든 부정적 예를 대상으로 모델을 학습시킬 시에는 엄청난 자원이 소모가 되기 때문에 적은 수의 부정적 예를 샘플링하는 기법을 사용함

- 네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실과 부정적 예 몇가지를 샘플링하여 구한 소실을 더한 값을 최종 손실로 봄

### 네거티브 샘플링의 샘플링 기법
- 부정적인 예를 샘플링하기 위해서 단순 무작위 추출하는 방법도 있지만 말뭉치의 통계 데이터를 기초로 고빈도 단어의 추추를 많이 하고 드문 단어는 적게 추출하는 가중치를 주어 샘플링하는 것이 좋음

- 그렇게 하기 위해 말뭉치(코퍼스)에서 각 단어의 출현 횟수를 구하여 이를 확률 분포로 나타낸 후 그 분포에 따라 단어를 샘플링할 수 있음


In [9]:
# 말뭉치에서의 단어 출현 빈도의 확률 분포에 따라 샘플링하는 예
import numpy as np

# 0 ~ 9까지의 숫자 중 하나를 무작위 샘플링
print(np.random.choice(10))

print(np.random.choice(10))

# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello','.']
print(np.random.choice(words))

# 5개만 무작위로 샘플링(중복허용)
print(np.random.choice(words, size=5, replace=False))

# 확률분포에 따른 샘플링
p =[0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))


8
5
hello
['goodbye' '.' 'you' 'I' 'hello']
say


In [12]:
# word2vec에서 네거티브 샘플링으로 확률분포를 사용할 시에는 기본 확률 분포에 0.75를 제곱하는 것을 권고함(꼭 0.75가 아니더라도 됨)
# 그 이유는 출현 확률이 낮은 단어를 버리지 않기 위해서임
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [13]:
# https://github.com/WegraLee/deep-learning-from-scratch-2/blob/master/ch04/negative_sampling_layer.py
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [ ]:
# 한 단어를 대상으로 확률분포를 만들어 네거비브 샘플링을 하는 클래스
class NegativeSamplimgLoss :
  def __init__(self, W, corpus, power=0.75, sample_size=5) :
    self.sample_size = sample_size
    self.sampler = UnigramSampler(corpus, power, sample_data)
    self.loss_layers = [SigmoidWtihLoss() for _ in range(sample_size + 1)]
    self.embed_dot_layers = [EmbeddingsDot(W) for _ in range(sample_size + 1)]
    self.parmas, self.grads = [], []

    for layer in self.embed_dot_layers :
      self.params += layer.params
      self.grads += layer.grads

    def forward(self, h , target) :
      batch_size = target.shape[0]
      negative_sample = self.sampler.get_negative_sample(target)

      # 긍정적 예 순전파
      score = self.embed_dot_layers[0].forward(h, target)
      correct_label = np.ones(batch_size, dtype=np.int32)
      loss = self.loss_layers[0].forward(score, correct_label)

      # 부정적 예 순전파
      negative_label = np.zeros(batch_size, dtype=np.int32)

      for i in range(self.sample_size) :
        negative_target = negative_sample[:, i]
        score = self.embed_dot_layers[1+i].forward(h, negative_target)
        loss += self.loss_layers[1+i].forward(score, negative_label)

      return loss

    def backward(self, dout=1) :
      dh = 0
      for l0, l1 in zip(self.loss_layers, self.embed_dot_layers) :
        dscore = l0.backward(dout)
        dh += l1.backward(dscore)

      return dh

In [15]:
# 개선된 CBOW 모델
class CBOW :
  def __init__(self, vocab_size, hidden_size, window_size, corpus) :
    V, H  = vocab_size, hidden_size

    # 가중치 초기화
    W_in = 0.01 * np.random.randn(V, H).astype('f')
    W_out = 0.01 * np.random.randn(V, H).astype('f')

    # 계층 생성
    self.in_layers = []

    for i in ragne(2 * window_size) :
      layer = Embedding(W_in)
      self.in_layers.append(layer)
    
    self.ns_loss = NegativeSamplimgLoss(W_out, corpus, power=0.75, sample_size=5)
    
    # 모든 가중치와 기울기를 배열에 모음
    layers = self.in_layers + [self.ns_loss]
    self.params, self.grads = [], []
    for layer in layers :
      self.params += layer.params
      self.grads += layer.grads

    # 인스턴스 변수에 단어의 분산 표현을 저장
    self.word_vecs = W_in

    def forward(self, contexts, target) :
      h = 0
      for i, layer in enumerate(self.in_layers) :
        h += layer.forward(contexts[:,i])
      
      h *= 1 / len(self.in_layers)
      loss = self.ns_loss.forward(h, target)
      return loss

    def backward(self, dout=1) :
      dout = self.ns_loss.backward(dout)
      dout *= 1 / len(self.in_layers)
      for layer in self.in_layers :
        layer.backward(dout)

      return None

In [ ]:
import numpy as np
from common import config
import pickle
# GPU 실행 시
# config.GPU = True # 쿠파이 필요
from common.trainer import Trainer
from common.optimizer import Adam

# 다음 코드는 실행은 되지 않으나 직접 손으로 치고 싶어서 넣음
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train)
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU : 
  contexts, target = to_gpu(contexts), to_gpu(target)

 # 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU :
  word_vecs = to_gpu(word_vecs)

params = {}
parmas['word_vecs'] = word_vecs.astype(np.float16) 
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f :
  pickle.dump(params, f, -1)

In [ ]:
# 이렇게 학습시킨 모델을 평가하는데 예전에 구현한 most_similar

In [ ]:
# https://github.com/WegraLee/deep-learning-from-scratch-2/blob/master/common/util.py
# 유추 문제를 풀 수 있도록 구현한 함수
# 예 ) man:King = woman :? 
def analogy(a, b, c, word_to_id, id_to_word, word_matrix, top=5, answer=None):
    for word in (a, b, c):
        if word not in word_to_id:
            print('%s(을)를 찾을 수 없습니다.' % word)
            return

    print('\n[analogy] ' + a + ':' + b + ' = ' + c + ':?')
    a_vec, b_vec, c_vec = word_matrix[word_to_id[a]], word_matrix[word_to_id[b]], word_matrix[word_to_id[c]]
    query_vec = b_vec - a_vec + c_vec
    query_vec = normalize(query_vec)

    similarity = np.dot(word_matrix, query_vec)

    if answer is not None:
        print("==>" + answer + ":" + str(np.dot(word_matrix[word_to_id[answer]], query_vec)))

    count = 0
    for i in (-1 * similarity).argsort():
        if np.isnan(similarity[i]):
            continue
        if id_to_word[i] in (a, b, c):
            continue
        print(' {0}: {1}'.format(id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return